In [ ]:
import numpy

import pyibm

from helper import *

%matplotlib inline

In [ ]:
pyibm.__version__

In [ ]:
delta_kernel = pyibm.delta_peskin_2002
delta_kernel_size = 4

In [ ]:
config = dict(x=dict(start=-2.0, end=2.0, num_cells=20),
              y=dict(start=-2.0, end=2.0, num_cells=20))
grid = pyibm.GridBase(config=config)
grid

In [ ]:
gridc = pyibm.GridCellCentered(grid=grid)
gridx = pyibm.GridFaceX(grid=grid)
gridy = pyibm.GridFaceY(grid=grid)

In [ ]:
GHat = pyibm.assemble_GHat(gridc, gridx, gridy)
print_matrix_info(GHat)
plot_matrix(GHat);

In [ ]:
MHat = pyibm.assemble_MHat(gridx, gridy)
print_matrix_info(MHat)
plot_matrix(MHat, figsize=(3.0, 3.0));

In [ ]:
G = MHat @ GHat
print_matrix_info(G)
plot_matrix(G);

In [ ]:
DHat = pyibm.assemble_DHat(gridc, gridx, gridy)
print_matrix_info(DHat)
plot_matrix(DHat);

In [ ]:
R = pyibm.assemble_R(gridx, gridy)
print_matrix_info(R)
plot_matrix(R, figsize=(3.0, 3.0));

In [ ]:
RInv = pyibm.assemble_RInv(R)
print_matrix_info(RInv)
plot_matrix(RInv, figsize=(3.0, 3.0));

In [ ]:
D = DHat @ RInv
print_matrix_info(D)
plot_matrix(D);

In [ ]:
# Check if divergence of the opposite of the transpose of the gradient.
A = D + G.T
A = A.multiply(A > 1e-12)
assert A.nnz == 0

In [ ]:
radius = 0.5
xc, yc = 0.0, 0.0
ds = gridx.x.get_widths()[0]
N = int(round(2 * numpy.pi * radius / ds))
epsilon = 0.0
theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1] + epsilon
x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
body = pyibm.Body(x, y, grid=gridc)
body

In [ ]:
Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                          kernel=delta_kernel,
                          kernel_size=delta_kernel_size)
print_matrix_info(Op)
plot_matrix(Op);

In [ ]:
S = pyibm.assemble_surfaces(body)
print_matrix_info(S)
plot_matrix(S, figsize=(3.0, 3.0));

In [ ]:
E = Op @ R @ MHat
print_matrix_info(E)
plot_matrix(E);

In [ ]:
H = Op.T @ S
print_matrix_info(H)
plot_matrix(H);

In [ ]:
ux = numpy.ones(gridx.size)
uy = numpy.zeros(gridy.size)
u = numpy.concatenate((ux, uy))
Ub = E @ u
print(Ub.shape)
print(Ub)

In [ ]:
Fx = numpy.ones(body.size)
Fy = numpy.zeros(body.size)
F = numpy.empty(body.ndim * body.size)
F[::body.ndim], F[1::body.ndim] = Fx, Fy

f = H @ F
fx = f[:gridx.size].reshape(gridx.shape)
fy = f[gridx.size:].reshape(gridy.shape)

F2 = E @ f
Fx2, Fy2 = F2[::body.ndim], F2[1::body.ndim]

dS = body.ds
dx = gridc.x.get_widths()[gridc.ijk(body.neighbors[0])[0]]
dy = gridc.y.get_widths()[gridc.ijk(body.neighbors[0])[1]]
dV = dx * dy

print(numpy.sum(Fx) * dS, numpy.sum(Fy) * dS)
print(numpy.sum(fx) * dV, numpy.sum(fy) * dV)
print(numpy.sum(Fx2) * dS, numpy.sum(Fy2) * dS)

In [ ]:
plot_contourf(fx, gridx, body, show_grid=False);

In [ ]:
Re = 100.0
LHat = 1 / Re * pyibm.assemble_LHat(gridx, gridy)
L = MHat @ LHat @ RInv
print_matrix_info(L)
plot_matrix(L);

In [ ]:
dt = 0.01
alpha_implicit = 0.5
M = MHat @ RInv
A = 1 / dt * M - alpha_implicit * L
print_matrix_info(A)
plot_matrix(A);

In [ ]:
MHatInv = pyibm.assemble_MHatInv(MHat)
MInv = R @ MHatInv
BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=1, L=L, MInv=MInv)
print_matrix_info(BN)
plot_matrix(BN);

In [ ]:
DBNG = D @ BN @ G
print_matrix_info(DBNG)
plot_matrix(DBNG);

In [ ]:
EBNH = E @ BN @ H
print_matrix_info(EBNH)
plot_matrix(EBNH);

In [ ]:
numpy.linalg.cond(EBNH.todense())

In [ ]:
Nmax = 3
for N in range(1, Nmax + 1):
    BN_test = pyibm.assemble_BN(gridx, gridy, dt=dt,
                                N=N, L=L, MInv=MInv)
    DBNG_test = D @ BN_test @ G
    cond = numpy.linalg.cond(DBNG_test.todense())
    print(N, cond)

In [ ]:
Nmax = 3
for N in range(1, Nmax + 1):
    BN_test = pyibm.assemble_BN(gridx, gridy, dt=dt,
                                N=N, L=L, MInv=MInv)
    EBNH_test = E @ BN_test @ H
    cond = numpy.linalg.cond(EBNH_test.todense())
    print(N, cond)

In [ ]:
for r in [1.0, 2.0, 4.0]:
    ds = gridx.x.get_widths()[0] / r
    Nb = int(round(2 * numpy.pi * radius / ds))
    theta = numpy.linspace(0.0, 2 * numpy.pi, num=Nb + 1)[:-1]
    x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
    body_test = pyibm.Body(x, y, grid=gridc)
    Op_test = pyibm.assemble_delta(body_test, gridc, gridx, gridy,
                                   kernel=delta_kernel,
                                   kernel_size=delta_kernel_size)
    S_test = pyibm.assemble_surfaces(body_test)
    E_test = Op_test @ R @ MHat
    H_test = Op_test.T @ S_test
    Nmax = 3
    for N in range(1, Nmax + 1):
        BN_test = pyibm.assemble_BN(gridx, gridy, dt=dt,
                                    N=N, L=L, MInv=MInv)
        EBNH_test = E_test @ BN_test @ H_test
        cond = numpy.linalg.cond(EBNH_test.todense())
        print(r, N, cond)